<p style="text-align: center">
    <img src="../../assets/images/untref-logo-negro.svg" style="height: 50px;" />
</p>

<h3 style="text-align: center">Estructuras de Datos</h3>

<h2 style="text-align: center">Clase 7: Recorridos de grafos</h3>

## ¿Para qué?

- Encontrar todos los vértices alcanzables desde un vértice dado (por ejemplo, para encontrar que cosas puede eliminar un _garbage collector_).
- Encontrar el mejor camino de un vértice a otro (para tomar decisiones sobre qué ruta tomar).
- Averiguar si un grafo tiene ciclos.
- Y unas cuantas cosas más...

## Algoritmos

- Recorrido a lo ancho o BFS (_Breadth First Search_)
- Recorrido en profundidad DFS (_Depth First Search_)

### Recorrido a lo ancho - BFS

Para grafos no dirigidos.

#### Algoritmo

```
BFS (s: Vertice):
    q <- Cola()

    q.encolar(s)
    visitado[s] = True

    MIENTRAS NO q.esta_vacia()
        v = q.desencolar()

        PARA CADA w EN v.adyacentes:
            SI NO visitado[w]:
                visitado[w] = True
                q.encolar(w)
```

#### Complejidad

$$
\begin{align}
\mathcal{O}\left(\left|V\right|+\left|A\right|\right)
\end{align}
$$

#### Ejemplo

In [ ]:
from edd.cola import Cola
from edd.grafo import Grafo, Vertice


def bfs_por_pasos(self, s: Vertice):
    visitado = {}
    q = Cola()

    orden = []  ### IGNORAR

    q.encolar(s)
    visitado[s] = True
    orden.append(s.id)

    aristas = []  ### IGNORAR
    yield {
        "msj": f"Encolamos y visitamos {s.id} como nodo inicial",
        "q": q,
        "visitados": [u.id for u in visitado],
        "aristas": aristas,
        "orden": orden,
    }  ### IGNORAR

    while not q.esta_vacia():
        v = q.desencolar()

        yield {
            "msj": f"Desencolamos {v.id} y vamos a buscar sus adyacentes",
            "q": q,
            "visitados": [u.id for u in visitado],
            "aristas": aristas,
            "orden": orden,
        }  ### IGNORAR

        for w in v.adyacentes:
            aristas.append((v.id, w.id))  ### IGNORAR

            if not visitado.get(w, False):
                q.encolar(w)
                visitado[w] = True
                orden.append(w.id)
                msj = f"Desde {v.id}, llegamos a {w.id}, lo encolamos y visitamos"
            else:
                msj = f"Desde {v.id}, llegamos a {w.id}, pero ya está visitado"

            yield {
                "msj": msj,
                "q": q,
                "visitados": [u.id for u in visitado],
                "aristas": aristas,
                "orden": orden,
            }  ### IGNORAR

    yield {
        "msj": f"No quedan nodos encolados",
        "q": q,
        "visitados": [u.id for u in visitado],
        "aristas": aristas,
        "orden": orden,
    }  ### IGNORAR


# Hacemos "monkey patching" del método que acabamos de implementar.
Grafo.bfs_por_pasos = bfs_por_pasos

In [ ]:
from edd.grafo import Grafo

G = Grafo()
G.agregar_arista("A", "C")
G.agregar_arista("A", "B")
G.agregar_arista("A", "D")
G.agregar_arista("A", "F")
G.agregar_arista("C", "D")
G.agregar_arista("D", "G")
G.agregar_arista("B", "E")
G.agregar_arista("F", "G")
G.agregar_arista("E", "H")
G.agregar_arista("E", "J")
G.agregar_arista("G", "K")


pasos = G.bfs_por_pasos(G["A"])

In [ ]:
# Cada vez que se ejecute esta celda, se mostrará una iteración del algoritmo BFS.
try:
    estado = next(pasos)
except StopIteration:
    print("~ Fin ~\n")
finally:
    print(f">>> {estado['msj']}\n")
    print(f"q = {estado['q']}\n")
    print(f"Recorrido = {estado['orden']}")
    G.draw(
        highlight_edges=estado["aristas"],
        highlight_nodes=estado["visitados"],
        pos={
            "A": (2.5, 1),
            "B": (1, 0.5),
            "C": (2, 0.5),
            "D": (3, 0.5),
            "F": (4, 0.5),
            "E": (1.5, 0),
            "G": (3.5, 0),
            "H": (1.5, -0.5),
            "J": (2.5, -0.5),
            "K": (3.5, -0.5),
        },
    )

#### Aplicaciones 

##### Camino Mínimo en grafos sin pesos

<table style="width: 100%;">
    <tbody>
        <tr>
            <td><pre>BFS (s: Vertice):
    q <- Cola()<br><br><br><br>
    q.encolar(s)
    visitado[s] = True<br>
    MIENTRAS NO q.esta_vacia()
        v = q.desencolar()<br>
        PARA CADA w EN v.adyacentes:
            SI NO visitado[w]:
                visitado[w] = True<br><br>
                q.encolar(w)
</pre></td>
            <td><pre style="color: darkgray;"><span style="color: purple; font-weight: bold;">CAMINO_MINIMO_BFS</span> (s: Vertice)
    q <- Cola()<br>
    <span style="color: purple; font-weight: bold;">distancia[s] = 0
    previo[s] = None</span><br>
    q.encolar(s)
    visitado[s] = True<br>
    MIENTRAS NO q.esta_vacia()
        v = q.desencolar()<br>
        PARA CADA w EN v.adyacentes:
            SI NO visitado[w]:
                visitado[w] = True
                <span style="color: purple; font-weight: bold;">distancia[w] = distancia[v] + 1
                previo[w] = v</span>
                q.encolar(w)
</pre></td>
        </tr>
    </tbody>
</table>

##### Grafo Bipartito

> **Definición**
>
> Un grafo **no dirigido** es bipartito si los vértices se pueden dividir en dos grupos, de modo tal que las aristas vayan siempre de un vértice de un grupo a un vértice del otro grupo (por ejemplo las aristas definen relaciones entre alumnos y cursos).

<p style="text-align:center;">
    <img src="figuras/grafo-bipartito.png" style="width:500px;" />
</p>

Como ejemplo, podemos pensar que las aristas definen relaciones entre alumnos y cursos.

<p style="text-align:center;">
    <img src="figuras/grafo-bipartito-ordenado.png" style="width:500px;" />
</p>

<table style="width: 100%;">
    <tbody>
        <tr>
            <td><pre>BFS (s: Vertice):
    q <- Cola()<br><br><br>
    q.encolar(s)
    visitado[s] = True<br>
    MIENTRAS NO q.esta_vacia()
        v = q.desencolar()<br>
        PARA CADA w EN v.adyacentes:
            SI NO visitado[w]:
                visitado[w] = True<br>
                q.encolar(w)<br><br><br><br><br>
</pre></td>
            <td><pre style="color: darkgray;"><span style="color: purple; font-weight: bold;">ES_BIPARTITO</span> (s: Vertice):
    q <- Cola()<br>
    <span style="color: purple; font-weight: bold;">color[s] = True</span><br>
    q.encolar(s)
    visitado[s] = True<br>
    MIENTRAS NO q.esta_vacia()
        v = q.desencolar()<br>
        PARA CADA w EN v.adyacentes:
            SI NO visitado[w]:
                visitado[w] = True
                <span style="color: purple; font-weight: bold;">color[w] = NOT color[v]</span>
                q.encolar(w)
            <span style="color: purple; font-weight: bold;">SINO:
                SI color[w] == color[v]:
                    DEVOLVER False<br>
    DEVOLVER True</span>
</pre></td>
        </tr>
    </tbody>
</table>

##### Otras Aplicaciones

- **Web Crawler**
    - Bot que utilizan los motores de búsqueda para descubrir páginas siguiendo los enlaces que hay en ella.
- **Sistemas de navegación GPS**
    - Para encontrar localizaciones vecinas.
- **Analizar la propagación de información en redes sociales**

### Más definiciones

#### Grafo conexo

Un grafo no dirigido es conexo si para todo par de vértices $u$ y $v$ de $G$, hay un camino que los une.

#### Componentes conexas

Subgrafos conexos maximales de un grafo no dirigido.

### Recorrido en profundidad - DFS

#### Algoritmo

```
DFS (v, visitado = {}, contador = 0):
    visitado[v] = True
    contador += 1

    PARA CADA w EN v.adyacentes:
        SI NO visitado[w]:
            DFS(w, visitado, contador)

    DEVOLVER contador
```

> Si `contador < |V|` el grafo no es conexo. Para seguir visitando el resto de los vértices, tomar uno no visitado $u$ y llamar a `DFS(u)` (este procedimiento se deberá repetir con tantas componentes conexas como tenga el grafo).

#### Complejidad

$$
\begin{align}
\mathcal{O}\left(\left|V\right|+\left|A\right|\right)
\end{align}
$$

#### Ejemplo

In [ ]:
from edd.grafo import Grafo, Vertice


def dfs_por_pasos(self, v: Vertice, visitado={}, contador=0, aristas=[], orden=[], arbol=[]):
    visitado[v] = True
    contador += 1
    orden.append(v.id)

    yield {
        "msj": f"Visitamos {v.id}",
        "v": v,
        "visitados": [u.id for u in visitado],
        "aristas": aristas,
        "arbol": arbol,
        "orden": orden,
    }  ### IGNORAR

    for w in v.adyacentes:
        aristas.append((v.id, w.id))  ### IGNORAR

        if not visitado.get(w, False):
            arbol.append((v.id, w.id))
            yield {
                "msj": f"Desde {v.id}, llegamos a {w.id}. Y ahora inicialmos un nuevo DFS de {w.id}",
                "v": v,
                "visitados": [u.id for u in visitado],
                "aristas": aristas,
                "arbol": arbol,
                "orden": orden,
            }  ### IGNORAR

            yield from self.dfs_por_pasos(w, visitado, contador, aristas, orden, arbol)
        else:
            yield {
                "msj": f"Desde {v.id}, llegamos a {w.id}, pero ya está visitado",
                "v": v,
                "visitados": [u.id for u in visitado],
                "aristas": aristas,
                "arbol": arbol,
                "orden": orden,
            }  ### IGNORAR


# Hacemos "monkey patching" del método que acabamos de implementar.
Grafo.dfs_por_pasos = dfs_por_pasos

In [ ]:
from edd.grafo import Grafo

G = Grafo()
G.agregar_arista("A", "C")
G.agregar_arista("A", "B")
G.agregar_arista("A", "D")
G.agregar_arista("A", "F")
G.agregar_arista("C", "D")
G.agregar_arista("D", "G")
G.agregar_arista("B", "E")
G.agregar_arista("F", "G")
G.agregar_arista("E", "H")
G.agregar_arista("E", "J")
G.agregar_arista("G", "K")


pasos = G.dfs_por_pasos(G["A"])

In [ ]:
# Cada vez que se ejecute esta celda, se mostrará una iteración del algoritmo DFS.
try:
    estado = next(pasos)
except StopIteration:
    print("~ Fin ~\n")
    print(">>> Árbol DFS\n")
    print(f"Recorrido = {estado['orden']}")

    G_ = Grafo()
    for v, w in estado["arbol"]:
        G_.agregar_arista(v, w)
    G_.draw(
        pos={
            "A": (2.5, 1),
            "B": (1, 0.5),
            "C": (2, 0.5),
            "D": (3, 0.5),
            "F": (4, 0.5),
            "E": (2, 0),
            "G": (3, 0),
            "H": (1.5, -0.5),
            "J": (2.5, -0.5),
            "K": (3.5, -0.5),
        },
    )
finally:
    print(f">>> {estado['msj']}\n")
    print(f"v = {estado['v']}\n")
    print(f"Recorrido = {estado['orden']}")

    G.draw(
        highlight_edges=estado["aristas"],
        highlight_nodes=estado["visitados"],
        pos={
            "A": (2.5, 1),
            "B": (1, 0.5),
            "C": (2, 0.5),
            "D": (3, 0.5),
            "F": (4, 0.5),
            "E": (2, 0),
            "G": (3, 0),
            "H": (1.5, -0.5),
            "J": (2.5, -0.5),
            "K": (3.5, -0.5),
        },
    )

#### Aplicaciones

##### Componentes conexas

> Dado un grafo no dirigido $G$ nos puede interesar saber cuántas componentes conexas hay. Una componentes conexa es un conjunto de vértices tal que empezando en uno de ellos cualquiera podemos acceder al resto recorriendo las aristas.

```
COMPONENTES_CONEXAS (G: Grafo):
    PARA CADA v EN G.vertices:
        visitado[v] = -1

    contador = 0

    PARA CADA v EN G.vertices:
        SI visitado[v] == -1
            DFS(v, visitado, contador)
            contador += 1


DFS (v, visitado, contador):
    visitado[v] = contador
    PARA CADA w EN v.adyacentes:
        SI visitado[w] == -1:
            DFS(w, visitado, contador)
```

###### Ejemplo

In [ ]:
from edd.grafo import Grafo, Vertice


def componentes_conexas(self):
    visitado: dict[Vertice, int] = {}
    contador: int = 0

    for v in self.vertices:
        visitado[v] = -1

    for v in self.vertices:
        if visitado[v] == -1:
            dfs(v, visitado, contador)
            contador += 1

    return visitado


def dfs(u: Vertice, visitado: dict[Vertice, int], contador: int):
    visitado[u] = contador

    for w in u.adyacentes:
        if visitado[w] == -1:
            dfs(w, visitado, contador)


# Hacemos "monkey patching" del método que acabamos de implementar.
Grafo.componentes_conexas = componentes_conexas

In [ ]:
from edd.grafo import Grafo

G = Grafo()
G.agregar_arista("2", "4")
G.agregar_arista("2", "7")
G.agregar_arista("5", "11")
G.agregar_arista("4", "7")
G.agregar_arista("7", "8")
G.agregar_arista("7", "10")
G.agregar_arista("3", "6")
G.agregar_arista("1", "7")
G.agregar_arista("6", "12")
G.agregar_arista("7", "9")

G.draw(layout="graphviz")

In [ ]:
grupos = G.componentes_conexas()

node_labels = {v.id: grupo for v, grupo in grupos.items()}

G.draw(layout="graphviz", node_labels=node_labels)

##### Otras Aplicaciones

- Encontrar caminos en laberintos
- Encontrar componentes fuertemente conexas